In [16]:
import pymongo
from pymongo import MongoClient

In [17]:
client = MongoClient('mongo-csgy-6513-summer.db',
                      username='jwb10028',
                      password="bigdata",
                      authSource = "bigdata")
db=client.jwb10028
db
col = db.test
myquery = {}
mydoc = col.find(myquery)
for x in mydoc: print(x) 

In [18]:
db

Database(MongoClient(host=['mongo-csgy-6513-summer.db:27017'], document_class=dict, tz_aware=False, connect=True, authsource='bigdata'), 'jwb10028')

In [19]:
import pandas as pd
import json

# Load the Restaurants CSV into MongoDB
restaurants_df = pd.read_csv('data/Restaurants_in_Durham_County_NC.csv', delimiter=';')
restaurants_collection = db['restaurants']
restaurants_data = restaurants_df.to_dict(orient='records')
restaurants_collection.insert_many(restaurants_data)

# Load the Foreclosures JSON into MongoDB
with open('data/durham-nc-foreclosure-2006-2016.json') as file:
    foreclosure_data = json.load(file)
    if isinstance(foreclosure_data, list):
        db.foreclosures.insert_many(foreclosure_data)
    else:
        db.foreclosures.insert_one(foreclosure_data)



In [20]:
# Step 2: Find the center point
query = {
    'Rpt_Area_Desc': 'Food Service',
    'Seats': {'$gte': 100}
}
restaurants_cursor = restaurants_collection.find(query)
print(restaurants_cursor[0])

{'_id': ObjectId('66bba826f1bab79263bb2211'), 'ID': 58123, 'Premise_Name': 'BROOKDALE DURHAM IFS', 'Premise_Address1': '4434 BEN FRANKLIN BLVD', 'Premise_Address2': nan, 'Premise_City': 'DURHAM', 'Premise_State': 'NC', 'Premise_Zip': '27704', 'Premise_Phone': '(919) 479-9966', 'Hours_Of_Operation': nan, 'Opening_Date': '2003-10-15', 'Closing_Date': nan, 'Seats': 350.0, 'Water': '5 - Municipal/Community', 'Sewage': '3 - Municipal/Community', 'Insp_Freq': 4, 'Est_Group_Desc': 'Nursing Home', 'Risk': 4, 'Smoking_Allowed': 'NO', 'Type_Description': '16 - Institutional Food Service', 'Rpt_Area_Desc': 'Food Service', 'Status': 'ACTIVE', 'Transitional_Type_Desc': 'FOOD', 'geolocation': '36.0467802, -78.8895483'}


In [21]:
# Print the 'geolocation' field for inspection
for doc in restaurants_cursor:
    print(doc['geolocation'])
    break  # Just print the first one to inspect

36.0467802, -78.8895483


In [22]:
lats = []
lons = []

for doc in restaurants_cursor:
    geolocation = doc['geolocation']
    
    if isinstance(geolocation, str):
        
        lat, lon = map(float, geolocation.split(', '))
    elif isinstance(geolocation, float):
        
        lat, lon = geolocation, None  
        continue  
    else:
        # If it's a tuple/list, assume it's already split into lat/lon
        lat, lon = geolocation
    
    lats.append(lat)
    lons.append(lon)

# Calculate center point
center_lat = sum(lats) / len(lats)
center_lon = sum(lons) / len(lons)
center_point = [center_lon, center_lat]

print(f"Center point (Longitude, Latitude): {center_point}")

Center point (Longitude, Latitude): [-78.92125451207154, 35.969243700163126]


In [23]:
from pprint import pprint
# Convert foreclosure data to GeoJSON format
foreclosures_collection = db['foreclosures']

# Print one of the foreclosure documents to inspect its structure
foreclosure_sample = foreclosures_collection.find_one()
pprint(foreclosure_sample)


{'_id': ObjectId('66becbfa66469365576485b4'),
 'datasetid': 'foreclosure-2006-2016',
 'fields': {'address': '217 E CORPORATION ST',
            'geocode': [36.0013755, -78.8922549],
            'parcel_number': '110138',
            'year': '2006'},
 'geometry': {'coordinates': [-78.8922549, 36.0013755], 'type': 'Point'},
 'record_timestamp': '2017-03-06T12:41:48-05:00',
 'recordid': '629979c85b1cc68c1d4ee8cc351050bfe3592c62'}


In [24]:
# Update foreclosure documents to have a GeoJSON formatted 'location' field
foreclosures_cursor = foreclosures_collection.find()
print(foreclosures_cursor)

In [25]:
from pymongo import GEOSPHERE

for doc in foreclosures_cursor:
    # Check if the 'geocode' field exists in the document
    if 'geocode' in doc['fields']:
        lat, lon = doc['fields']['geocode']  # Access the geocode field directly
        geojson_point = {
            "type": "Point",
            "coordinates": [lon, lat]  # GeoJSON expects [longitude, latitude]
        }
        foreclosures_collection.update_one(
            {"_id": doc['_id']},
            {"$set": {"location": geojson_point}}
        )
    else:
        print(f"No geocode field found in document with _id: {doc['_id']}")

# Ensure 'location' field is indexed for geospatial queries
foreclosures_collection.create_index([("location", GEOSPHERE)])


No geocode field found in document with _id: 66becbfa66469365576486ae
No geocode field found in document with _id: 66becbfa66469365576486b2
No geocode field found in document with _id: 66becbfa6646936557648878
No geocode field found in document with _id: 66becbfa66469365576488e5
No geocode field found in document with _id: 66becbfa6646936557648b21
No geocode field found in document with _id: 66becbfa6646936557648bc0


'location_2dsphere'

In [26]:
# Define the center point and the radius (in meters)
center_point_geojson = {"type": "Point", "coordinates": center_point}
radius = 3218.69  # 2 miles in meters

# Perform the geospatial query
nearby_foreclosures = foreclosures_collection.find({
    "location": {
        "$near": {
            "$geometry": center_point_geojson,
            "$maxDistance": radius
        }
    }
})



In [27]:
# Convert the cursor to a list and then count the number of documents
foreclosure_list = list(nearby_foreclosures)
foreclosure_count = len(foreclosure_list)

print(f"Number of foreclosures within 2 miles: {foreclosure_count}")


Number of foreclosures within 2 miles: 524


In [28]:
# Count the total number of documents in the collection
total_foreclosures = foreclosures_collection.count_documents({})

print(f'Total number of foreclosures: {total_foreclosures}')

Total number of foreclosures: 1948


In [29]:
#foreclosures_collection.delete_many({})